# DAT600


### Ardijan Rexhaj, Daniel Alvsåker, Ove Oftedal


## Task 1:

### a\) 1.

1 -> [2] <br>
2 -> [2, 3, 4] <br>
3 -> [1, 2, 5] <br>
4 -> [5, 6] <br>
5 -> [3, 4] <br>
6 -> [4] <br>


### a\) 2.

```
     1◄───┐
     │    │
     ▼    │
  ┌─►2◄──►3
  └──┤    ▲
     │    │
     ▼    ▼
     4◄──►5
     ▲
     │
     ▼
     6
```

### a\) 3.





